In [ ]:
import torch
import torchvision
from torchvision import transforms

# 加载测试集
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)

# 定义攻击函数
def fgsm_attack(image, epsilon, data_grad):
    # 计算扰动值
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    # 限制像素值范围在[0,1]
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# 定义测试函数
def test(model, device, test_loader, epsilon):
    correct = 0
    adv_examples = []
    for data, target in test_loader:
        # 找到被分类正确的图像
        data, target = data.to(device), target.to(device)
        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]
        if init_pred.item() != target.item():
            continue
        correct += 1
        # 对该图像进行攻击
        data.requires_grad = True
        output = model(data)
        loss = torch.nn.functional.nll_loss(output, target)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data
        perturbed_data = fgsm_attack(data, epsilon, data_grad)
        # 将生成的对抗样本加入列表
        adv_examples.append((data, perturbed_data, target))
        # 打印进度信息
        if len(adv_examples) >= 1000:
            break
        if len(adv_examples) % 100 == 0:
            print(f"Attack progress: {len(adv_examples)}/{1000}")
    # 计算攻击成功率
    final_acc = correct / float(len(test_loader))
    print(f"Correctly classified examples: {correct}/{len(test_loader)}")
    print(f"Attack success rate: {(1000-correct)/1000:.4f}")
    return adv_examples

# 设置攻击参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epsilon = 0.1

# 对模型进行测试和攻击
model.eval()
adv_examples = test(model, device, test_loader, epsilon)
